# Categorical Transformation for DL
> a list of things to categoryical transformation

In [2]:
# default_exp category

## Imports

In [3]:
# export
import pandas as pd
import numpy as np
from pathlib import Path
import json

In [27]:
# export
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from typing import Iterable

class C2I:
    def __init__(
        self,
        arr:Iterable,
        pad_mst:bool=False,
    ):
        self.pad_mst = pad_mst
        self.pad = ["[MST]",] if self.pad_mst else []
        self.dict = dict(
            (v,k) for k,v in enumerate(self.pad + list(arr)))
        self.get_int = self.get_get_int()
        self.get_int_ = np.vectorize(self.get_int)
        
    def get_get_int(self,):
        if self.pad_mst:
            def get_int(idx:str) -> int:
                if idx in self.dict:
                    return self.dict[idx]
                else:
                    return 0
        else:
            def get_int(idx:str) -> int:
                return self.dict[idx]
        return get_int

    def __repr__(self) -> str:
        return f"C2I:{self.__len__()} categories"

    def __len__(self):
        return len(self.dict)

    def __getitem__(self, k:int):
        if type(k) == np.ndarray:
            return self.get_int_(k)
        else:
            return self.get_int(k)
        
class Category:
    def __init__(
        self,
        arr:Iterable,
        pad_mst:bool=False
    ):
        self.pad_mst=pad_mst
        self.c2i = C2I(arr, pad_mst=pad_mst)
        self.i2c = np.array(self.c2i.pad+list(arr))

    def save(self,path: Path):
        with open(path,"w") as f:
            json.dump(self.i2c.tolist(),f)

    @classmethod
    def load(cls, path:Path, pad_mst:bool=False):
        with open(path,"r") as f:
            l = np.array(json.load(f))
        return cls(l, pad_mst=pad_mst)

    def __len__(self):
        return len(self.i2c)

    def __repr__(self):
        return f"Category Manager with {self.__len__()}"

## Test effect

In [21]:
cates = Category(range(50))

In [26]:
test_c = np.random.randint(0,30,1000)
(cates.c2i[test_c]==test_c).mean()

1.0